###Imports

In [3]:
import numpy as np
from numpy import *
from sympy import Derivative, diff, simplify
from sympy import Symbol
from sympy import *
import sympy
import time
from math import log

###Implementación de los métodos DFP y BFGS

In [6]:
#armijo: num num num func np.array(2x1) np.array(2x1) num -> num
#Esta función entrega un alfa apropiado según la condición de armijo
#dado=
#a0: un alfa inicial. 
#t: el tau de la condición
#w: el omega de la condición
#f: la función objetivo
#xk: el punto de la iteración
#dk: la dirección de la iteración
#valor: la función objetivo evaluada en xk

def armijo(a0, t, w, f, xk, dk, valor):
  
  contador=0
  ak=a0

  auxAR=sympy.sympify(f).subs(x,xk[0]+ak*dk[0])
  valor1=sympy.sympify(auxAR).subs(y,xk[1]+ak*dk[1])
  
  #cálculo del gradiente evaluado en xk
  gradAux1=np.array([(f.diff(x)),(f.diff(y))])
  aux2=(sympy.sympify(gradAux1).subs(x,xk[0]))
  aux3=(sympy.sympify(aux2).subs(y,xk[1]))
  gradiente=np.array([[aux3[0]],[aux3[1]]])
  
  if valor1<=valor+(w*ak)*gradiente.T.dot(dk):
    return ak
  else:
    ak=t*ak
    contador+=1
    return armijo(ak, t, w, f, xk, dk, valor)

In [7]:
#DFP: np.array(2x2) np.array(2x1) np.array(2x1) num -> np.array(2x2)
#Entrega la siguiente matriz del método DFP,dada la matriz anterior
#y las variables pk, qk ,yk del método 

def DFP(H,pk,qk,yk):
  HNueva=H-H.dot(qk.dot(qk.T)).dot(H)/(qk.T.dot(H.dot(qk)))+yk*pk.dot(pk.T)
  return HNueva

In [8]:
#minDFP: func np.array(2x1) num int num num-> np.array(4x1)
#minimiza la función objetivo con el método DFP dada la función, un punto inicial,
#un valor épsilon de tolerancia, el número de iteraciones máximas permitidas,
#un alfa inicial para armijo y un omega para armijo

def minDFP(f, x0, e, iteracionesMax, a0, w):
  contador=0
  H=np.eye(2)
  xk=x0
  tiempoInicial = time.time()
  while contador<iteracionesMax:
    #función evaluada en xk
    aux1=(sympy.sympify(f).subs(x,xk[0]))
    valor=(sympy.sympify(aux1).subs(y,xk[1]))

    #cálculo del gradiente evaluado en xk
    gradAux1=np.array([(f.diff(x)),(f.diff(y))])
    aux2=(sympy.sympify(gradAux1).subs(x,xk[0]))
    aux3=(sympy.sympify(aux2).subs(y,xk[1]))
    gradiente=np.array([[aux3[0]],[aux3[1]]])

    if gradiente[0]**2+gradiente[1]**2<e**2:
      tiempoFinal=time.time()
      duracion=tiempoFinal-tiempoInicial
      return np.array([[xk],[valor],[contador],[duracion]])
    
    #dirección del paso
    dk=-H.dot(gradiente)
    
    #alfa (armijo(a0, t, w, f, xk, dk, valor))
    ak=armijo(a0, 0.4, w, f, xk, dk, valor)

    #cálculo del siguiente punto
    xSiguiente=xk+ak*dk

    #cálculo del siguiente gradiente
    gradAux2=np.array([(f.diff(x)),(f.diff(y))])
    aux4=(sympy.sympify(gradAux2).subs(x,xSiguiente[0]))
    aux5=(sympy.sympify(aux4).subs(y,xSiguiente[1]))
    gradSiguiente=np.array([[aux5[0]],[aux5[1]]])

    #cálculo de la siguiente H
    pk=xSiguiente-xk
    qk=gradSiguiente-gradiente
    yk=1/pk.T.dot(qk)
    
    #nueva H
    H=DFP(H,pk,qk,yk)

    #actualizar datos
    xk=xSiguiente
    contador+=1
  print("Iteraciones máximas superadas")
  return

In [9]:
#BFGS: np.array(2x2) np.array(2x1) np.array(2x1) num -> np.array(2x2)
#Entrega la siguiente matriz del método BFGS,dada la matriz anterior
#y las variables pk, qk ,yk del método 

def BFGS(H,pk,qk,yk):
  Id=np.eye(2)
  HNueva=(Id-yk*pk.dot(qk.T)).dot(H.dot(Id-yk*qk.dot(pk.T)))+yk*pk.dot(pk.T)
  return HNueva

In [10]:
#minBFGS: func np.array(2x1) num int num num-> np.array(4x1)
#minimiza la función objetivo con el método BFGS dada la función, un punto inicial,
#un valor épsilon de tolerancia, el número de iteraciones máximas permitidas,
#un alfa inicial para armijo y un omega para armijo

def minBFGS(f, x0, e, iteracionesMax, a0, w):
  contador=0
  H=np.eye(2)
  xk=x0
  tiempoInicial = time.time()
  while contador<iteracionesMax:
    #función evaluada en xk
    aux1=(sympy.sympify(f).subs(x,xk[0]))
    valor=(sympy.sympify(aux1).subs(y,xk[1]))

    #cálculo del gradiente evaluado en xk
    gradAux1=np.array([(f.diff(x)),(f.diff(y))])
    aux2=(sympy.sympify(gradAux1).subs(x,xk[0]))
    aux3=(sympy.sympify(aux2).subs(y,xk[1]))
    gradiente=np.array([[aux3[0]],[aux3[1]]])

    if gradiente[0]**2+gradiente[1]**2<e**2:
      tiempoFinal=time.time()
      duracion=tiempoFinal-tiempoInicial
      return np.array([[xk],[valor],[contador],[duracion]])
    

    #dirección del paso
    dk=-H.dot(gradiente)
    
    #alfa (armijo(a0, t, w, f, xk, dk, valor de f en xk))
    ak=armijo(a0, 0.4, w, f, xk, dk, valor)

    #cálculo del siguiente punto
    xSiguiente=xk+ak*dk
    
    #cálculo del siguiente gradiente
    gradAux2=np.array([(f.diff(x)),(f.diff(y))])
    aux4=(sympy.sympify(gradAux2).subs(x,xSiguiente[0]))
    aux5=(sympy.sympify(aux4).subs(y,xSiguiente[1]))
    gradSiguiente=np.array([[aux5[0]],[aux5[1]]])
    

    #cálculo de la siguiente H
    pk=xSiguiente-xk
    qk=gradSiguiente-gradiente
    yk=1/pk.T.dot(qk)
    
    #nueva H
    H=BFGS(H,pk,qk,yk)
    
    #actualizar datos
    xk=xSiguiente
    contador+=1
  print("Iteraciones máximas superadas")
  return

###P1 Uso de los métodos en las funciones

####Función 1

In [ ]:
x=Symbol('x')
y=Symbol('y')
f1=x**2 + y**2
print("Metodo DFP")
sol1=minDFP(f1, np.array([[-24.],[23.]]), 0.1, 1000, 1, 0.0001)
print("Punto ideal:")
print(sol1[0][0])
print("Valor:")
print(sol1[1])
print("Iteraciones:")
print(sol1[2])
print("Tiempo demorado (s):")
print(sol1[3])

print("")

print("Metodo BFGS")
sol2=minBFGS(f1, np.array([[-24],[23.]]), 0.1, 1000, 1, 0.0001)

print("Punto ideal:")
print(sol2[0][0])
print("Valor:")
print(sol2[1])
print("Iteraciones:")
print(sol2[2])
print("Tiempo demorado (s):")
print(sol2[3])

Metodo DFP
Punto ideal:
[[0]
 [-8.88178419700125e-16]]
Valor:
[7.88860905221012e-31]
Iteraciones:
[2]
Tiempo demorado (s):
[0.015521764755249023]

Metodo BFGS
Punto ideal:
[[0]
 [0]]
Valor:
[0]
Iteraciones:
[2]
Tiempo demorado (s):
[0.008841276168823242]


####Función 2


In [ ]:
x=Symbol('x')
y=Symbol('y')
f2= (x+2*y-7)**2 + (2*x+y-5)**2
print("Metodo DFP")
sol1=minDFP(f2, np.array([[-24.],[23.]]), 0.1, 1000, 1, 0.0001)
print("Punto ideal:")
print(sol1[0][0])
print("Valor:")
print(sol1[1])
print("Iteraciones:")
print(sol1[2])
print("Tiempo demorado (s):")
print(sol1[3])

print("")

print("Metodo BFGS")
sol2=minBFGS(f2, np.array([[-24],[23.]]), 0.1, 1000, 1, 0.0001)

print("Punto ideal:")
print(sol2[0][0])
print("Valor:")
print(sol2[1])
print("Iteraciones:")
print(sol2[2])
print("Tiempo demorado (s):")
print(sol2[3])

Metodo DFP
Punto ideal:
[[1.00044755727725]
 [2.99957914496850]]
Valor:
[3.80278513674318e-7]
Iteraciones:
[4]
Tiempo demorado (s):
[0.05342531204223633]

Metodo BFGS
Punto ideal:
[[1.00034614623616]
 [3.00252815484470]]
Valor:
[3.95578109494544e-5]
Iteraciones:
[4]
Tiempo demorado (s):
[0.029596805572509766]


####Función 3

In [ ]:
x=Symbol('x')
y=Symbol('y')
f3= (x**2 +y-11)**2 + (x + y**2 - 7)**2
print("Metodo DFP")
sol1=minDFP(f3, np.array([[-24.],[23.]]), 0.1, 1000, 1, 0.0001)
print("Punto ideal:")
print(sol1[0][0])
print("Valor:")
print(sol1[1])
print("Iteraciones:")
print(sol1[2])
print("Tiempo demorado (s):")
print(sol1[3])

print("")

print("Metodo BFGS")
sol2=minBFGS(f3, np.array([[-24],[23.]]), 0.1, 1000, 1, 0.0001)

print("Punto ideal:")
print(sol2[0][0])
print("Valor:")
print(sol2[1])
print("Iteraciones:")
print(sol2[2])
print("Tiempo demorado (s):")
print(sol2[3])

Metodo DFP
Punto ideal:
[[3.58381910510125]
 [-1.84676900579577]]
Valor:
[4.06999929252785e-5]
Iteraciones:
[35]
Tiempo demorado (s):
[0.27361059188842773]

Metodo BFGS
Punto ideal:
[[3.58455529796635]
 [-1.84809099698903]]
Valor:
[8.94343347640957e-7]
Iteraciones:
[17]
Tiempo demorado (s):
[0.15492749214172363]


###P2 Método Barrera

In [11]:
#barrera: func func np.array num int num num
#Resuelve un problema de minimización dada
#la función objetivo, la función de restricción,
#un punto inicial factible, un epsilon de tolerancia,
#la cantidad máxima de subproblemas a realizar
#y los valores de alfa y omega para el paso de armijo.

def barrera(f,g,x0,e,subproblemasMax, a0, w):
  contador=0
  tiempoInicial = time.time()
  b=-sympy.log(-g)
  tk=1/(contador+1)**3
  while contador<subproblemasMax:

    tksig=1/(contador+2)**3
    iteracion=minBFGS(f+tk*b, x0, 0.1, subproblemasMax, a0, w)

    xk=iteracion[0][0]

    xksig=minBFGS(f+tksig*b, x0, 0.1, subproblemasMax, a0, w)[0][0]

    nxk=sqrt(xk[0]**2 + xk[1]**2)
    nxksig=sqrt(xksig[0]**2 + xksig[1]**2)
    
    if nxksig-nxk <= e:
      sol=iteracion

      #cálculo duración
      tiempoFinal = time.time()
      duracion=tiempoFinal-tiempoInicial

      #cálculo del valor en la solución
      aux1=sympy.sympify(f).subs(x,sol[0][0][0])
      valor=sympy.sympify(aux1).subs(y,sol[0][0][1])

      print("Punto ideal:")
      print(sol[0][0])
      print("Valor:") 
      print(valor)
      print("Subproblemas resueltos:")
      print(contador+1)
      print("Tiempo demorado (s):")
      print(duracion)
      return

    #actualizar datos
    tk=tksig
    contador+=1
    
  print('Cantidad de subproblemas máximos superada')

####Función 1

In [ ]:
x=Symbol('x')
y=Symbol('y')
f1=x**2 + y**2
g1=x+y+100

#barrera(f,g,x0,e,subproblemasMax, a0, w)
barrera(f1, g1, np.array([[-100.],[-70.]]), 0.1, 10000000, 0.001, 0.0001)

Punto ideal:
[[-50.0111697664278]
 [-49.9988361928255]]
Valor:
5001.00072204346
Subproblemas resueltos:
1
Tiempo demorado (s):
144.85149717330933


####Función 2

In [ ]:
#9 horas.
#Este problema demoró alrededor de 9 horas. 

x=Symbol('x')
y=Symbol('y')
f2=(1-x)**2 + 100*(y-x**2)**2
g2=(x-1)**3-y+1

#barrera(f,g,x0,e,subproblemasMax, a0, w)
barrera(f2, g2, np.array([[-10.],[ -10.]]), 0.1, 10000000, 0.00001, 0.0000000000001)

Punto ideal:
[[-0.506263011142664]
 [0.258124543788796]]
Valor:
2.26916033913982
Subproblemas resueltos:
1
Tiempo demorado (s):
32576.222229242325


####Función 3

In [ ]:
#Esta parte demoró mucho, por lo que no arroja solución.

#x=Symbol('x')
#y=Symbol('y')
#f3=(1-x)**2 + 100*(y-x**2)**2
#g3=x+y-2

#barrera(f,g,x0,e,subproblemasMax, a0, w)
#barrera(f3, g3, np.array([[-10.],[-10.]]), 0.1, 10000000, 0.000005, 0.00000000000001)